In [1]:
# Exploring the role of regularization and Dropout

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
# Load the pickle file

pickle_file = '/Users/rgparekh/Documents/Personal/Rajesh/Data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
# Reshape the datasets into a flat vector

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
# Accuracy calculation
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [16]:
# Training a logistic regression classifier with L2 loss using TensorFlow

# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

# Regularization parameter (lower C for stronger regularization)
C = 50.0 

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
    
  # Apply the L2 loss
  logits = tf.matmul(tf_train_dataset, weights) + biases
  # TODO: Should we regularize the bias terms?
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
    1/C * tf.nn.l2_loss(weights))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [17]:
# Let's run the Tensor Flow graph performing GD

num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Loss at step 0: 75.109474
Training accuracy: 11.3%
Validation accuracy: 14.9%
Loss at step 100: 8.432969
Training accuracy: 76.0%
Validation accuracy: 74.4%
Loss at step 200: 1.678783
Training accuracy: 82.4%
Validation accuracy: 80.5%
Loss at step 300: 0.861463
Training accuracy: 83.6%
Validation accuracy: 81.7%
Loss at step 400: 0.759820
Training accuracy: 83.8%
Validation accuracy: 81.7%
Loss at step 500: 0.746803
Training accuracy: 83.9%
Validation accuracy: 81.7%
Loss at step 600: 0.745043
Training accuracy: 83.9%
Validation accuracy: 81.7%
Loss at step 700: 0.744761
Training accuracy: 83.9%
Validation accuracy: 81.7%
Loss at step 800: 0.744691
Training accuracy: 83.9%
Validation accuracy: 81.7%
Test accuracy: 88.7%


In [18]:
# Stochastic Gradient Descent (SGD) with L2 loss

batch_size = 512

# Regularization parameter (lower C for stronger regularization)
C = 50.0 

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  # TODO: Should we regularize the bias terms?
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
    1/C * tf.nn.l2_loss(weights))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [19]:
# Run the SGD network (with L2 regularization)

num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 75.346817
Minibatch accuracy: 9.2%
Validation accuracy: 13.7%
Minibatch loss at step 500: 0.792599
Minibatch accuracy: 81.4%
Validation accuracy: 81.5%
Minibatch loss at step 1000: 0.792955
Minibatch accuracy: 81.4%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 0.718559
Minibatch accuracy: 84.4%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 0.893087
Minibatch accuracy: 78.5%
Validation accuracy: 81.9%
Minibatch loss at step 2500: 0.827276
Minibatch accuracy: 80.7%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 0.797821
Minibatch accuracy: 82.4%
Validation accuracy: 81.8%
Minibatch loss at step 3500: 0.839977
Minibatch accuracy: 79.1%
Validation accuracy: 82.0%
Minibatch loss at step 4000: 0.738418
Minibatch accuracy: 83.8%
Validation accuracy: 82.1%
Minibatch loss at step 4500: 0.738068
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Minibatch loss at step 5000: 0.935112
Minibatch accuracy: 76.4%
Validation accurac

In [30]:
# Construct a multi-layer perceptron

# Constants
image_size = 28
num_labels = 10

# Regularization parameter (lower C for stronger regularization)
C = 100.0 

# Parameters
learning_rate = 0.001
batch_size = 1024

graph = tf.Graph()
with graph.as_default():

    # Network Parameters
    n_hidden_1 = 256 # Hidden layer 1
    n_hidden_2 = 128 # Hidden layer 2 

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                  shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Create model
    def multilayer_perceptron(x, weights, biases):
        # Hidden layer 1 with RELU activation
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)
        # Hidden layer 2 with RELU activation
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = tf.nn.relu(layer_2)
        # Output layer with linear activation
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
        return out_layer

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([image_size*image_size, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_hidden_2, num_labels]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([num_labels]))
    }

    # Construct model
    pred = multilayer_perceptron(tf_train_dataset, weights, biases)

    # Define loss and optimizer
    # loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=tf_train_labels))
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=tf_train_labels) + 
        1/C * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['h2']) + tf.nn.l2_loss(weights['out'])))

    # GradientDescentOptimizer is the standard GD
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    
    #AdamOptimizer is an optimized version of GD
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(pred)
    valid_prediction = tf.nn.softmax(multilayer_perceptron(valid_dataset, weights, biases))
    test_prediction = tf.nn.softmax(multilayer_perceptron(test_dataset, weights, biases))

# Run the network
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")

  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))



Initialized
Minibatch loss at step 0: 2872.452393
Minibatch accuracy: 8.5%
Validation accuracy: 7.1%
Minibatch loss at step 500: 1141.667603
Minibatch accuracy: 75.6%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 923.889038
Minibatch accuracy: 76.6%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 726.248291
Minibatch accuracy: 70.0%
Validation accuracy: 68.7%
Minibatch loss at step 2000: 548.020142
Minibatch accuracy: 64.4%
Validation accuracy: 63.9%
Minibatch loss at step 2500: 393.512878
Minibatch accuracy: 62.7%
Validation accuracy: 63.0%
Minibatch loss at step 3000: 266.645386
Minibatch accuracy: 68.8%
Validation accuracy: 67.7%
Minibatch loss at step 3500: 170.006180
Minibatch accuracy: 74.3%
Validation accuracy: 72.4%
Minibatch loss at step 4000: 101.674225
Minibatch accuracy: 76.5%
Validation accuracy: 74.8%
Minibatch loss at step 4500: 57.871414
Minibatch accuracy: 80.4%
Validation accuracy: 79.1%
Minibatch loss at step 5000: 31.950785
Minibatch accuracy: 82.2

In [14]:
# Multi-layer perceptron with dropout

# Construct a multi-layer perceptron

# Constants
image_size = 28
num_labels = 10

# Regularization parameter (lower C for stronger regularization)
C = 100.0 

# Parameters
learning_rate = 0.005
batch_size = 256

# Dropout keep_probability
keep_prob = 1.0

graph = tf.Graph()
with graph.as_default():

    # Network Parameters
    n_hidden_1 = 128 # Hidden layer 1
    n_hidden_2 = 64 # Hidden layer 2 

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                  shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Dropout
    keep_probability = tf.placeholder(tf.float32)
  
    # Create model
    def multilayer_perceptron(x, weights, biases, keep_probability):
        # Hidden layer 1 with RELU activation
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)
        layer_1 = tf.nn.dropout(layer_1, keep_probability)
        
        # Hidden layer 2 with RELU activation
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = tf.nn.relu(layer_2)
        layer_2 = tf.nn.dropout(layer_2, keep_probability)
        
        # Output layer with linear activation
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
        return out_layer

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([image_size*image_size, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_hidden_2, num_labels]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([num_labels]))
    }

    # Construct model
    pred = multilayer_perceptron(tf_train_dataset, weights, biases, keep_prob)

    # Define loss and optimizer
    # loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=tf_train_labels))
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=tf_train_labels) + 
        1/C * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['h2']) + tf.nn.l2_loss(weights['out'])))

    # GradientDescentOptimizer is the standard GD
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    
    #AdamOptimizer is an optimized version of GD
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(pred)
    valid_prediction = tf.nn.softmax(multilayer_perceptron(valid_dataset, weights, biases, 1.0))
    test_prediction = tf.nn.softmax(multilayer_perceptron(test_dataset, weights, biases, 1.0))

# Run the network
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")

  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 1525.894653
Minibatch accuracy: 9.0%
Validation accuracy: 11.3%
Minibatch loss at step 500: 280.598511
Minibatch accuracy: 55.5%
Validation accuracy: 56.2%
Minibatch loss at step 1000: 91.945587
Minibatch accuracy: 64.5%
Validation accuracy: 66.6%
Minibatch loss at step 1500: 25.988920
Minibatch accuracy: 77.0%
Validation accuracy: 75.8%
Minibatch loss at step 2000: 8.680697
Minibatch accuracy: 85.9%
Validation accuracy: 81.8%
Minibatch loss at step 2500: 4.124907
Minibatch accuracy: 82.4%
Validation accuracy: 83.1%
Minibatch loss at step 3000: 2.404408
Minibatch accuracy: 80.9%
Validation accuracy: 83.8%
Minibatch loss at step 3500: 1.480975
Minibatch accuracy: 85.2%
Validation accuracy: 84.2%
Minibatch loss at step 4000: 1.089345
Minibatch accuracy: 85.2%
Validation accuracy: 83.9%
Minibatch loss at step 4500: 0.953209
Minibatch accuracy: 85.5%
Validation accuracy: 83.8%
Minibatch loss at step 5000: 0.899803
Minibatch accuracy: 80.9%
Validation a